In [1]:
import os
import json
import pandas as pd
from PyPDF2 import PdfReader
import traceback

In [2]:
import sys
print(sys.executable)

c:\Users\PMLS\envs\myenv\python.exe


In [7]:
from langchain.chat_models import ChatOpenAI,openai
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [ ]:
import os
llm=ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.7,  #(0-2(creative))
    max_tokens=2000  
)

# prompt=PromptTemplate(
#     imput_variables=['country'],
#     template="tell me the national sport of the coutry {country}"
# )
# chain=LLMChain(llm=client,prompt=prompt,memory=memory)

# response = chain.invoke({"country": "Pakistan"})
# print(response["text"])



ImportError: Could not import openai python package. Please install it with `pip install openai`.